In [2]:
from matplotlib import pyplot as plt
from environment_wrapper import *
import networkx as nx
from numpy.random import randint

env = GraphWrapper()
running_reward = 0
for e in range(1):

    env.reset()
    state = env.observe()
    done = False
    episode_reward = 0
    actions = 0
    while not done:
        G, node_inputs, leaf_nodes = state 
        if len(env.leaf_nodes) > 0:
            node = env.leaf_nodes[randint(len(env.leaf_nodes))]
        else :
            node = 0
        state, reward, done = env.step((node, 1), False)
        episode_reward += reward
        actions += 1

    # agent.save(version="2")
    running_reward = 0.05 * episode_reward + (1 - 0.05) * running_reward
    print(actions, episode_reward)

    # log results
    if e % 1 == 0:
        print('Episode {}\tLast reward: {:.2f}\tAverage reward: {:.2f} \n'.format(
            e, episode_reward, running_reward))


1293 -892.1854999999999
Episode 0	Last reward: -892.19	Average reward: -44.61 

